In [ ]:
%%capture
!pip install git+https://github.com/vyperlang/titanoboa.git@master

In [ ]:
from google.colab import userdata
import boa

## **Setting Up the Infrastructure**

Before we can simulate some transactions, we need to initialize the necessary contract. For API keys, we can use the "Secrets" feature of Google Colab. An `RPC_ETHEREUM` including an HTTP key (e.g., from Alchemy) and an `ETHERSCAN_API_KEY` including an Etherscan API key need to be created before the notebook can be used.

Interacting with the contract is very simple when using titanoboa. The `from_etherscan` function lets us connect to the contracts.

`boa.env.fork` forks the desired chain for us.

Official Titanoboa Documentation: [https://titanoboa.readthedocs.io/en/latest/](https://titanoboa.readthedocs.io/en/latest/)


In [ ]:
boa.env.fork(userdata.get('RPC_ETHEREUM'))

ONEWAY_LENDING_FACTORY = '0xc67a44D958eeF0ff316C3a7c9E14FB96f6DedAA3'

factory = boa.from_etherscan(
    address=ONEWAY_LENDING_FACTORY,
    name='OneWayLendingFactory',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

factory

### **Borrow Rates**

Rate values are denominated in seconds and have a base unit of 1e18.

Minimum and maximum borrow rates for lending markets default to `min_default_borrow_rate` and `max_default_borrow_rate` if input values for `min_borrow_rate` and `max_borrow_rate` are set to zero. They can be changed by the admin.

If custom values are used, they need to be within the range of `MIN_RATE` and `MAX_RATE`. These values are the absolute minimum and maximum rates, which are hardcoded and cannot be changed.

First, let's check the current rates:

In [ ]:
print("MIN_RATE:", factory.MIN_RATE())
print("MAX_RATE:", factory.MAX_RATE())
print("min_default_borrow_rate", factory.min_default_borrow_rate())
print("max_default_borrow_rate", factory.max_default_borrow_rate())

*To calculate the annual rates, one needs to multiply the rate by the number of seconds per year:*

In [ ]:
print("MIN_RATE:", (factory.MIN_RATE() / 1e18) * 86400*365)
print("MAX_RATE:", (factory.MAX_RATE() / 1e18) * 86400*365)
print("min_default_borrow_rate", (factory.min_default_borrow_rate() / 1e18) * 86400*365)
print("max_default_borrow_rate", (factory.max_default_borrow_rate() / 1e18) * 86400*365)

## **Changing Default Borrow Rates**

Only the admin of the contract can change the default borrow rates. The default borrow rates are not of great importance, as users can specify their own rates when creating lending markets. Nonetheless, let's change them:

### **`set_default_rates`**

`OneWayLendingVaultFactory.set_default_rates(min_rate: uint256, max_rate: uint256):`

Function to set new values for the maximum (`max_default_borrow_rate`) and minimum (`min_default_borrow_rate`) default borrow rates.

| Input      | Type      | Description                       |
|------------|-----------|-----------------------------------|
| `min_rate` | `uint256` | New minimum default borrow rate.  |
| `max_rate` | `uint256` | New maximum default borrow rate.  |


In [ ]:
ADMIN = factory.admin()

with boa.env.prank(ADMIN):
    factory.set_default_rates(317097918, 23782343987)

assert factory.min_default_borrow_rate() == 317097918
assert factory.max_default_borrow_rate() == 23782343987

print("New minimum default borrow rate:",factory.min_default_borrow_rate())
print("New maximum default borrow rate:", factory.max_default_borrow_rate())

In [ ]:
print("min_default_borrow_rate", (factory.min_default_borrow_rate() / 1e18) * 86400*365)
print("max_default_borrow_rate", (factory.max_default_borrow_rate() / 1e18) * 86400*365)

**As shown, we successfully changed the minimum and maximum default borrow rates of the OneWay Lending Factory.**